In [2]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import IsolationForest
import optuna
import pandas as pd

d:\GitHub repos\ADL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
random_state = 42 #! ABSOLUTELY DO NOT CHANGE THIS VALUE AS IT WILL CHANGE THE TEST/HOLDOUT SET
raw_dataset = pd.read_csv("./data/processed_data_OHE.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# Slice your data


X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)
HOLDOUT_DATA = pd.concat([X_FINAL_TEST, Y_FINAL_TEST], axis=1)

FOLDS_DATA = pd.concat([X_FOR_FOLDS, Y_FOR_FOLDS], axis=1)
FOLDS_DATA.describe()

,Age,Gender,UAlb,Ucr,UACR,TC,TG,TCTG,LDLC,HDLC,...,Community_chonggu,Community_huaxin,Community_jinze,Community_liantang,Community_xianghuaqiao,Community_xujin,Community_yingpu,Community_zhaoxian,Community_zhujiajiao,DR
count,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,...,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000
mean,63.800766,0.549983,49.823755,4873.045280,71.036695,5.348884,1.842762,4.404497,3.217346,1.334314,...,0.072449,0.110589,0.083595,0.104145,0.076106,0.088645,0.125914,0.106583,0.114072,0.101010
std,7.483781,0.497539,124.383230,6063.166596,1140.344876,1.131424,2.191222,2.654190,0.953018,0.312538,...,0.259252,0.313649,0.276803,0.305475,0.265191,0.284256,0.331782,0.308609,0.317926,0.301368
min,34.000000,0.000000,0.100000,0.000000,0.100000,2.100000,0.100000,0.550000,0.220000,0.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,59.000000,0.000000,5.300000,7.000000,5.800000,4.622500,0.920000,2.540000,2.570000,1.110000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,64.000000,1.000000,12.300000,17.000000,12.150000,5.270000,1.380000,3.810000,3.200000,1.290000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,69.000000,1.000000,33.100000,9223.750000,33.275000,5.980000,2.090000,5.690000,3.820000,1.520000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,92.000000,1.000000,1065.000000,22726.000000,85427.700000,25.000000,82.400000,26.820000,8.640000,3.180000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
folds Generator, oversample first, synthetic later(outlier_detection=True, n_splits=5, shuffle=True)


if outlier_detection:
    do OD AND REMOVAL
    
oversample= Smotenc
oversample + original training
synthetic = synthesizer(oversample + original training)

print similarity scores between original and synthetic data

scaler = StandardScaler()

save training and test data for each fold, automatic numbering convention